In [12]:
import sklearn
import pandas as pd
from influxdb import InfluxDBClient
from sklearn import manifold, random_projection
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from metrics.pyDRMetrics import DRMetrics
import matplotlib.pyplot as plt
import yaml

In [1]:
with open('/home/min/Multi-level-Feature-Driven-Storage-Server/configuration_manager/config.yaml','r') as yamlfile:
    data = yaml.load(yamlfile,Loader=yaml.FullLoader)

DBclient = InfluxDBClient(host=data['global']['database_ip'], port=data['global']['database_port'], database=data['global']['database_name'], username='root', password='root')

max_ = 0
min_ = 0

# Note that the input values to PCA should not be negative or the principle would be different because of the order 
table_name = "visual_features_entropy_unnormalized"
result = DBclient.query('SELECT * FROM '+table_name)
result_point = pd.DataFrame(result.get_points(measurement=table_name))
meta_X = result_point['name']

X = result_point.drop(columns = ['name', 'time'])


isomap = manifold.Isomap(n_neighbors=12, n_components=1)
isomap_y = isomap.fit_transform(X)

pca = PCA(n_components=1)
pca_y = pca.fit_transform(X)

drm_isomap = DRMetrics(X, isomap_y)
drm_pca = DRMetrics(X, pca_y)




In [14]:
rps_transformer = random_projection.SparseRandomProjection(n_components = 1)
rps_y = rps_transformer.fit_transform(X)
drm_rps = DRMetrics(X, rps_y)

rpg_transformer = random_projection.GaussianRandomProjection(n_components = 1)
rpg_y = rpg_transformer.fit_transform(X)
drm_rpg = DRMetrics(X, rpg_y)


In [15]:
# # minMax normalization
    # max_ = max(max_, y.max(axis=0))
# min_ = min(min_, y.min(axis=0))
# y_std = (y - min_) / (max_ - min_)

plt.plot(drm_pca.T, label = 'pca', color = 'b')
plt.plot(drm_isomap.T, label = 'isomap', color = 'y')
plt.plot(drm_rps.T, label = 'isomap', color = 'g')
plt.plot(drm_rpg.T, label = 'isomap', color = 'r')

plt.ylim(-0.1,1.1)
plt.legend()
plt.savefig('T.png')
plt.clf()

plt.plot(drm_pca.C, label = 'pca', color = 'b')
plt.plot(drm_isomap.C, label = 'isomap', color = 'y')
plt.plot(drm_rps.C, label = 'isomap', color = 'g')
plt.plot(drm_rpg.C, label = 'isomap', color = 'r')

plt.ylim(-0.1,1.1)
plt.legend()
plt.savefig('C.png')
plt.clf()

print('PCA T AUC = ', drm_pca.AUC_T)
print('iso T AUC = ', drm_isomap.AUC_T)
print('rps T AUC = ', drm_rps.AUC_T)
print('rpg T AUC = ', drm_rpg.AUC_T)

print('PCA C AUC = ', drm_pca.AUC_C)
print('iso C AUC = ', drm_isomap.AUC_C)
print('rps C AUC = ', drm_rps.AUC_C)
print('rpg C AUC = ', drm_rpg.AUC_C)

PCA T AUC =  0.9999272734462155
iso T AUC =  0.999908591737824
rps T AUC =  0.9995950063559146
rpg T AUC =  0.9992610652235363
PCA C AUC =  0.9999747947854137
iso C AUC =  0.9999523847035376
rps C AUC =  0.9996472703273783
rpg C AUC =  0.9994570828868007


<Figure size 432x288 with 0 Axes>

In [18]:
print("%.9f"% (drm_pca.Vr))
print("%.9f"% (drm_isomap.Vr))
print("%.9f"% (drm_rps.Vr))
print("%.9f"% (drm_rpg.Vr))

0.000068295
0.000118581
0.001649217
0.002947369
